In [ ]:
#pip install pca

In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm

In [56]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

**My main changes and what I want people to review in the code block below:**
    
1) I added an IV section

2) I added a new thing where we take half of the mismeasured covariate values and then use them as the exponent to raise e to (e.g. z4 = e**z4)

3) i got rid of the 1,000 value for beta1 and beta2

4) i got rid of the p = 10 value

**Comments on those changes:**

1) i'm curious if i did this right

2) i did this because of bonhomme's feedback about where PCA might outperform taking the simple avg. we've gotten some tentative signs that PCA does do better than the avg when you stretch out he observations

3) , 4) it seemed like we didn't really need those values

In [63]:
output = pd.DataFrame()
N = 2000

for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    for exp_of_var in ['yes','no']:
                        for k in tqdm(range(1000)):
                            # Initialize Lists
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Take e to the power of the values for half of the measurements if log_of_var is true
                            if exp_of_var == 'yes':
                                mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):] =np.exp(mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):])
                                
                            # Do feature scaling (normalize to mean 0 and variance 1)
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = pca()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results['PC']['PC1']

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Mismeasured Results
                            model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[0])

                            # All Variables Mismeasured Results
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                            # Average Mismeasured Variables Results
                            model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                            # PCA Results
                            model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[0])

                            # Instrumental Variables Results
                            z_string = mismeasured_z.columns[1]
                            for z in mismeasured_z.columns[2:]:
                                z_string = z_string + ' + ' + z
                            # Create the predicted value of z1
                            vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                            iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                            iv_coef.append(iv_results.params[0])

                            # True Results
                            model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[0])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['exp_of_var'] = exp_of_var
                            output = output.append(new_output)

output

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:17<00:00,  1.48it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.375025,0.261870,0.423040,0.275726,0.087581,0.303200,0.5,0.1,1.0,5,yes
0,0.387540,0.260679,0.450304,0.295075,0.092977,0.303639,0.5,0.1,1.0,5,yes
0,0.368100,0.271400,0.449494,0.309250,0.105595,0.329061,0.5,0.1,1.0,5,yes
0,0.341565,0.206938,0.462255,0.243375,0.017567,0.249387,0.5,0.1,1.0,5,yes
0,0.350082,0.251845,0.429354,0.271611,0.076058,0.304343,0.5,0.1,1.0,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,10.235847,10.044752,10.046002,10.045785,9.945562,10.061697,0.5,10.0,1.0,5,no
0,10.337028,10.127853,10.130077,10.130208,10.009207,10.143512,0.5,10.0,1.0,5,no
0,10.365423,10.176099,10.176876,10.176753,10.065601,10.190903,0.5,10.0,1.0,5,no
0,10.301485,10.109853,10.110269,10.110708,10.016573,10.125927,0.5,10.0,1.0,5,no


In [65]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.375025,0.261870,0.423040,0.275726,0.087581,0.303200,0.5,0.1,1.0,5,yes
0,0.387540,0.260679,0.450304,0.295075,0.092977,0.303639,0.5,0.1,1.0,5,yes
0,0.368100,0.271400,0.449494,0.309250,0.105595,0.329061,0.5,0.1,1.0,5,yes
0,0.341565,0.206938,0.462255,0.243375,0.017567,0.249387,0.5,0.1,1.0,5,yes
0,0.350082,0.251845,0.429354,0.271611,0.076058,0.304343,0.5,0.1,1.0,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,10.235847,10.044752,10.046002,10.045785,9.945562,10.061697,0.5,10.0,1.0,5,no
0,10.337028,10.127853,10.130077,10.130208,10.009207,10.143512,0.5,10.0,1.0,5,no
0,10.365423,10.176099,10.176876,10.176753,10.065601,10.190903,0.5,10.0,1.0,5,no
0,10.301485,10.109853,10.110269,10.110708,10.016573,10.125927,0.5,10.0,1.0,5,no


In [64]:
output.to_csv('C://Users//paulo//Documents//Econometrics and ML//Final Project//n_2000_results.csv')

In [28]:
pd.set_option('display.max_rows', 10)

In [67]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']
output['iv_coef_ape'] = (output['iv_coef'] - output['beta1']).abs()/output['beta1']

In [68]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.243860,0.539131,0.539201,0.538429,0.999531,0.487426,0.461571,0.756140,0.460869,0.460799,0.512574
1,-0.9,1.0,1.0,5,yes,0.243096,0.377095,0.177435,0.337474,1.000026,0.302393,0.662526,0.756904,0.622905,0.822565,0.697607
2,-0.5,1.0,1.0,5,no,0.713676,0.894460,0.894489,0.894244,1.000278,0.873928,0.105756,0.286324,0.105540,0.105511,0.126072
3,-0.5,1.0,1.0,5,yes,0.716052,0.819329,0.636681,0.793310,1.000516,0.767512,0.206690,0.283948,0.180671,0.363319,0.232488
4,0.0,1.0,1.0,5,no,0.999795,0.999899,0.999867,0.999851,0.999964,0.999946,0.019268,0.021877,0.019330,0.019264,0.019602
5,0.0,1.0,1.0,5,yes,0.997359,0.997289,0.997972,0.997402,0.997469,0.997549,0.020847,0.022893,0.020897,0.023182,0.021192
6,0.5,0.1,1.0,5,no,0.384428,0.204685,0.204708,0.204935,0.099550,0.225383,1.049350,2.844282,1.046855,1.047077,1.253834
7,0.5,0.1,1.0,5,yes,0.385096,0.281346,0.464594,0.307314,0.099823,0.333336,2.073135,2.850962,1.813463,3.645937,2.333358
8,0.5,1.0,0.1,5,no,1.028959,1.010672,1.010648,1.010679,1.000280,1.012703,0.022338,0.031997,0.022327,0.022320,0.023050
9,0.5,1.0,0.1,5,yes,1.028554,1.018286,1.036255,1.020779,1.000050,1.023371,0.026581,0.031992,0.025296,0.037743,0.027977


In [69]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.030085,0.039279,0.039244,0.039182,0.048965,0.037788,0.039182,0.030085,0.039279,0.039244,0.037788
1,-0.9,1.0,1.0,5,yes,0.031224,0.036622,0.030188,0.040780,0.051900,0.034748,0.040780,0.031224,0.036622,0.030188,0.034748
2,-0.5,1.0,1.0,5,no,0.028830,0.027048,0.027005,0.026986,0.025932,0.027272,0.026986,0.028830,0.027048,0.027005,0.027272
3,-0.5,1.0,1.0,5,yes,0.028971,0.028340,0.034268,0.032873,0.026112,0.029639,0.032873,0.028971,0.028340,0.034268,0.029639
4,0.0,1.0,1.0,5,no,0.027062,0.023950,0.023904,0.023915,0.022085,0.024384,0.014153,0.015915,0.014127,0.014139,0.014490
5,0.0,1.0,1.0,5,yes,0.028345,0.026173,0.028993,0.026306,0.023233,0.026505,0.016239,0.016906,0.015977,0.017514,0.016093
6,0.5,0.1,1.0,5,no,0.028235,0.026090,0.026056,0.026057,0.024890,0.026601,0.260571,0.282354,0.260902,0.260560,0.266009
7,0.5,0.1,1.0,5,yes,0.028551,0.027928,0.034165,0.032064,0.025999,0.028821,0.320635,0.285505,0.279277,0.341646,0.288206
8,0.5,1.0,0.1,5,no,0.024046,0.025430,0.025395,0.025403,0.026112,0.025317,0.016124,0.019819,0.016177,0.016117,0.016452
9,0.5,1.0,0.1,5,yes,0.024268,0.024862,0.023597,0.024722,0.025993,0.024367,0.018334,0.019508,0.017672,0.021133,0.018896


In [70]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,1.286016,1.181877,1.364840,1.208039,0.999941,1.233656,0.208039,0.286016,0.181877,0.364840,0.233656
1,0.5,1.0,1.0,20,1.286302,1.057247,1.303374,1.082180,1.000563,1.064656,0.082184,0.286302,0.057517,0.303374,0.064712
2,0.5,1.0,1.0,50,1.285761,1.024265,1.285302,1.054081,0.999367,1.039384,0.054642,0.285761,0.029253,0.285302,0.040854


In [72]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,0.029114,0.028548,0.033858,0.032386,0.025992,0.029468,0.032386,0.029114,0.028548,0.033858,0.029468
1,0.5,1.0,1.0,20,0.029351,0.026911,0.033285,0.028555,0.026149,0.027054,0.028543,0.029351,0.026328,0.033285,0.026920
2,0.5,1.0,1.0,50,0.028327,0.026355,0.032338,0.027981,0.025693,0.026280,0.026868,0.028327,0.020674,0.032338,0.023928


In [73]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.385096,0.281346,0.464594,0.307314,0.099823,0.333336,2.073135,2.850962,1.813463,3.645937,2.333358
1,0.5,1.0,1.0,5,1.286016,1.181877,1.364840,1.208039,0.999941,1.233656,0.208039,0.286016,0.181877,0.364840,0.233656
2,0.5,10.0,1.0,5,10.284043,10.179820,10.363310,10.205351,9.999421,10.231638,0.020535,0.028404,0.017982,0.036331,0.023164


In [74]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.028551,0.027928,0.034165,0.032064,0.025999,0.028821,0.320635,0.285505,0.279277,0.341646,0.288206
1,0.5,1.0,1.0,5,0.029114,0.028548,0.033858,0.032386,0.025992,0.029468,0.032386,0.029114,0.028548,0.033858,0.029468
2,0.5,10.0,1.0,5,0.028068,0.027471,0.033213,0.031681,0.024802,0.028296,0.003168,0.002807,0.002747,0.003321,0.002830


In [75]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,1.028554,1.018286,1.036255,1.020779,1.000050,1.023371,0.026581,0.031992,0.025296,0.037743,0.027977
1,0.5,1.0,1.0,5,1.286016,1.181877,1.364840,1.208039,0.999941,1.233656,0.208039,0.286016,0.181877,0.364840,0.233656
2,0.5,1.0,10.0,5,3.871104,2.826099,4.669066,3.090914,0.999783,3.347121,2.090914,2.871104,1.826099,3.669066,2.347121


In [76]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,0.024268,0.024862,0.023597,0.024722,0.025993,0.024367,0.018334,0.019508,0.017672,0.021133,0.018896
1,0.5,1.0,1.0,5,0.029114,0.028548,0.033858,0.032386,0.025992,0.029468,0.032386,0.029114,0.028548,0.033858,0.029468
2,0.5,1.0,10.0,5,0.157908,0.128864,0.247093,0.213507,0.025090,0.160145,0.213507,0.157908,0.128864,0.247093,0.160145


In [77]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.243096,0.377095,0.177435,0.337474,1.000026,0.302393,0.662526,0.756904,0.622905,0.822565,0.697607
1,-0.5,1.0,1.0,5,0.716052,0.819329,0.636681,0.793310,1.000516,0.767512,0.206690,0.283948,0.180671,0.363319,0.232488
2,0.0,1.0,1.0,5,0.997359,0.997289,0.997972,0.997402,0.997469,0.997549,0.020847,0.022893,0.020897,0.023182,0.021192
3,0.5,1.0,1.0,5,1.286016,1.181877,1.364840,1.208039,0.999941,1.233656,0.208039,0.286016,0.181877,0.364840,0.233656
4,0.9,1.0,1.0,5,1.756712,1.623982,1.822437,1.662590,1.000460,1.698283,0.662590,0.756712,0.623982,0.822437,0.698283


In [78]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.031224,0.036622,0.030188,0.040780,0.051900,0.034748,0.040780,0.031224,0.036622,0.030188,0.034748
1,-0.5,1.0,1.0,5,0.028971,0.028340,0.034268,0.032873,0.026112,0.029639,0.032873,0.028971,0.028340,0.034268,0.029639
2,0.0,1.0,1.0,5,0.028345,0.026173,0.028993,0.026306,0.023233,0.026505,0.016239,0.016906,0.015977,0.017514,0.016093
3,0.5,1.0,1.0,5,0.029114,0.028548,0.033858,0.032386,0.025992,0.029468,0.032386,0.029114,0.028548,0.033858,0.029468
4,0.9,1.0,1.0,5,0.030737,0.036026,0.029998,0.040690,0.051766,0.034581,0.040690,0.030737,0.036026,0.029998,0.034581


In [88]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,1.284098,1.103888,1.103891,1.104196,0.999016,1.124581,0.104196,0.284098,0.103888,0.103891,0.124581
1,0.5,1.0,1.0,20,no,1.284549,1.029979,1.029960,1.030268,0.998874,1.037325,0.033097,0.284549,0.032862,0.032844,0.038777
2,0.5,1.0,1.0,50,no,1.286865,1.013139,1.013170,1.013513,1.000362,1.028554,0.023088,0.286865,0.023286,0.022975,0.031933


In [90]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,0.028280,0.027487,0.027473,0.027465,0.026566,0.027945,0.027465,0.028280,0.027487,0.027473,0.027945
1,0.5,1.0,1.0,20,no,0.028618,0.026115,0.025949,0.025967,0.025786,0.026113,0.022245,0.028618,0.022375,0.022182,0.023902
2,0.5,1.0,1.0,50,no,0.028791,0.026077,0.025654,0.025659,0.025411,0.025897,0.017538,0.028791,0.017608,0.017417,0.021589


In [82]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.384428,0.204685,0.204708,0.204935,0.099550,0.225383,1.049350,2.844282,1.046855,1.047077,1.253834
1,0.5,1.0,1.0,5,1.284098,1.103888,1.103891,1.104196,0.999016,1.124581,0.104196,0.284098,0.103888,0.103891,0.124581
2,0.5,10.0,1.0,5,10.286165,10.105609,10.105663,10.105950,10.000385,10.126254,0.010595,0.028617,0.010561,0.010566,0.012625


In [83]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.028235,0.026090,0.026056,0.026057,0.024890,0.026601,0.260571,0.282354,0.260902,0.260560,0.266009
1,0.5,1.0,1.0,5,0.028280,0.027487,0.027473,0.027465,0.026566,0.027945,0.027465,0.028280,0.027487,0.027473,0.027945
2,0.5,10.0,1.0,5,0.029120,0.027425,0.027421,0.027444,0.026720,0.027706,0.002744,0.002912,0.002743,0.002742,0.002771


In [84]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,1.028959,1.010672,1.010648,1.010679,1.000280,1.012703,0.022338,0.031997,0.022327,0.022320,0.023050
1,0.5,1.0,1.0,5,1.284098,1.103888,1.103891,1.104196,0.999016,1.124581,0.104196,0.284098,0.103888,0.103891,0.124581
2,0.5,1.0,10.0,5,3.863389,2.054238,2.054027,2.056737,0.998997,2.258838,1.056737,2.863389,1.054238,1.054027,1.258838


In [85]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,0.024046,0.025430,0.025395,0.025403,0.026112,0.025317,0.016124,0.019819,0.016177,0.016117,0.016452
1,0.5,1.0,1.0,5,0.028280,0.027487,0.027473,0.027465,0.026566,0.027945,0.027465,0.028280,0.027487,0.027473,0.027945
2,0.5,1.0,10.0,5,0.159474,0.106256,0.106113,0.105870,0.025178,0.115603,0.105870,0.159474,0.106256,0.106113,0.115603


In [86]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.243860,0.539131,0.539201,0.538429,0.999531,0.487426,0.461571,0.756140,0.460869,0.460799,0.512574
1,-0.5,1.0,1.0,5,0.713676,0.894460,0.894489,0.894244,1.000278,0.873928,0.105756,0.286324,0.105540,0.105511,0.126072
2,0.0,1.0,1.0,5,0.999795,0.999899,0.999867,0.999851,0.999964,0.999946,0.019268,0.021877,0.019330,0.019264,0.019602
3,0.5,1.0,1.0,5,1.284098,1.103888,1.103891,1.104196,0.999016,1.124581,0.104196,0.284098,0.103888,0.103891,0.124581
4,0.9,1.0,1.0,5,1.758564,1.463140,1.463153,1.463880,1.002911,1.514488,0.463880,0.758564,0.463140,0.463153,0.514488


In [87]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.030085,0.039279,0.039244,0.039182,0.048965,0.037788,0.039182,0.030085,0.039279,0.039244,0.037788
1,-0.5,1.0,1.0,5,0.028830,0.027048,0.027005,0.026986,0.025932,0.027272,0.026986,0.028830,0.027048,0.027005,0.027272
2,0.0,1.0,1.0,5,0.027062,0.023950,0.023904,0.023915,0.022085,0.024384,0.014153,0.015915,0.014127,0.014139,0.014490
3,0.5,1.0,1.0,5,0.028280,0.027487,0.027473,0.027465,0.026566,0.027945,0.027465,0.028280,0.027487,0.027473,0.027945
4,0.9,1.0,1.0,5,0.030986,0.041232,0.041169,0.041208,0.050420,0.039861,0.041208,0.030986,0.041232,0.041169,0.039861
